In [2]:
import os
import sys
import time
import glob
import shutil
import numpy as np
import pandas as pd

import subprocess
from pathlib import Path
from subprocess import Popen, PIPE
import copy

from scipy import spatial
from scipy import ndimage
from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy.table import Table
from astropy.stats import mad_std
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
# from astropy.stats import SigmaClip
from astropy.wcs.utils import skycoord_to_pixel
# from photutils.background import Background2D, MedianBackground,SExtractorBackground
from photutils.centroids import centroid_1dg, centroid_2dg
import warnings

from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

from photutils.segmentation import detect_sources, deblend_sources



In [3]:
def create_mask(id_in_list, data_img, wht_img):

    ### want to read in the cutout image and then make segmentation map from it. ###
    segment_map = detect_sources(data_img, threshold=1.5*wht_img, npixels=20)
    #print(segment_map.shape)


    segm_deblend = deblend_sources(data_img, segment_map,
                               npixels=20, nlevels=32, contrast=0.001,
                               progress_bar=True)
    
    mask_cutout = segm_deblend
    # wcs_seg = WCS(data_img[0].header)

    from skimage.draw import disk

    #ny, nx = mask_cutout.shape
    ny,nx = 101,101
    ic, jc = ny//2, nx//2
    label = mask_cutout.data[ic, jc]
 
    mask_cutout = mask_cutout.data

    mask = np.where(mask_cutout == int(label))

    mask_cutout[mask] = 0
    # mask_cutout[~mask] = 0


    return np.array(mask_cutout)


In [13]:
def make_stamp(id_, img, sig, ra, dec, filt, psf_filename,seg=False):
    filepath = "/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/"

    folder = filepath + f"pysersic_files/{id_}/"

    stamp_name = f"{str(id_)}"+'_'+str(filt)+'_'+"stamp.fits"
    mask_name = f"{str(id_)}"+'_'+str(filt)+'_'+"mask.fits"

    if os.path.exists(folder):
        # print("Path already exists!")
      #     return
      # else:
          # os.makedirs(folder)

      ### for image stamp ###
        hdu=img
        wcs = WCS(hdu[0].header)
        size = (101, 101) #change to 60,60 for hst

        ### uncomment if need to use sky coords instead of pixel coords. ###
       
        coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, unit="deg")

        ## uncomment below for images which aren't in uJy. ###
        # old_ZP = -2.5*np.log10(hdu[0].header['PHOTFLAM']) - 5*np.log10(hdu[0].header['PHOTPLAM']) -2.408194
        # unit_to_uJy = 10**((23.9-old_ZP)/2.5)

        cutout = Cutout2D(hdu[0].data, coord, size, wcs =wcs)
        fits.writeto(os.path.join(folder, stamp_name), data=cutout.data, overwrite=True)#*unit_to_uJy add this back for hst
      
        ### for sigma map ###
        wcs_rms = WCS(img[0].header)
        rms_cutout = Cutout2D(sig[0].data, coord, size, wcs=wcs_rms).data
        rms_map = 1/np.sqrt(rms_cutout)
        fits.writeto(os.path.join(folder, str(id_)+'_'+str(filt)+'_'+"rms.fits"), data = rms_map, overwrite = True)#*unit_to_uJy
        data_img = cutout.data
        wht_img = rms_map

        mask_cutout = create_mask(id_,data_img,wht_img)
        fits.writeto(os.path.join(folder, mask_name), data = mask_cutout, overwrite = True)

        shutil.copy(psf_filename, folder)

        return 0

In [48]:
id_ = 'rubies_uds_144195'

ra,dec = 34.32515592884,-5.143685286

F115W_PSF = '/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/F115w_psf_norm.fits'
F200W_PSF = '/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/F200w_psf_norm.fits'
F444W_PSF = '/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/F444w_psf_norm.fits'

wht_img_F115W = fits.open('/Users/mhamadouche/PRIMER/primer_jwst_nircam_uds_f115w_v1.0_wht.fits')
sci_img_F115W = fits.open('/Users/mhamadouche/PRIMER/primer_jwst_nircam_uds_f115w_v1.0_sci.fits')

wht_img_F200W = fits.open('/Users/mhamadouche/PRIMER/primer_jwst_nircam_uds_f200w_v1.0_wht.fits')
sci_img_F200W = fits.open('/Users/mhamadouche/PRIMER/primer_jwst_nircam_uds_f200w_v1.0_sci.fits')

wht_img_F444W = fits.open('/Users/mhamadouche/PRIMER/primer_jwst_nircam_uds_f444w_v1.0_wht.fits')
sci_img_F444W = fits.open('/Users/mhamadouche/PRIMER/primer_jwst_nircam_uds_f444w_v1.0_sci.fits')

filters = ["F115W", "F200W", "F444W"]

imgs = [sci_img_F115W,sci_img_F200W, sci_img_F444W]
whts = [wht_img_F115W,wht_img_F200W,wht_img_F444W]
psfs = [F115W_PSF, F200W_PSF, F444W_PSF]

for i, filt in enumerate(filters):
    img = imgs[i]
    wht = whts[i]
    psf_name = psfs[i]

    make_stamp(id_, img, wht,ra, dec,filt, psf_filename=psf_name, seg=True)


Deblending: 0it [00:00, ?it/s]

Deblending:   0%|          | 0/2 [00:00<?, ?it/s]

Deblending:   0%|          | 0/10 [00:00<?, ?it/s]

In [49]:
F115W_PSF = 'F115w_psf_norm.fits'
F200W_PSF = 'F200w_psf_norm.fits'
F444W_PSF = 'F444w_psf_norm.fits'

psfs = [F115W_PSF,F200W_PSF,F444W_PSF]

In [ ]:
def create_galfit_file(id, filter, conf = True):
    ### basic input info ###

    input_gfile = f"{str(id)}_{str(filter)}_img_cutout.fits"
    output_gfile = f"{str(id)}_{str(filter)}_imgblock.fits"
    rms = f"{str(id)}_{str(filter)}_rms.fits"
    zp = 28.087
    filepath = f'/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/codes/_test/'

    folder =  filepath + f"{str(id)}/"
    psfs = [F115W_PSF, F200W_PSF, F444W_PSF]

    if filter == 'F115W':
        img = sci_img_F115W
        psf_filename = psfs[0]
    elif filter == 'F200W':
        img = sci_img_F200W
        psf_filename = psfs[1]
    elif filter == 'F444W':
        img = sci_img_F444W
        psf_filename = psfs[2]
    
    wcs = WCS(img[0].header)

    pixel_scale = 0.03

    ### define input parameters - taken from galfit columns for f356w ###
    # flux = ind.loc[str(id), "f150w"]  ## flux is in uJy
    # mag = ind.loc[str(id), 'mag']#
    mag = 22
    # mag = 23.9 - 2.5 * np.log10(flux)
    # reff = ind.loc[str(id), "re"]
    reff = 0.5
    # sersic = ind.loc[str(id), "ng"]
    sersic = 4.5
    #reff = ind.loc[str(id), "R_KRON"]
    # ratio = ind.loc[str(id), "q"] ### ellipticity is a/b, in galfit q is b/a so need to do 1-ratio in param file ###
    ratio = 0.8
    #ratio = ind.loc[str(id), "Q_1"]
    # angle = ind.loc[str(id), "pa"]
    angle = 45
    # x,y = 30.5, 30.5 ### for hst ###
    x,y = 101/2, 101/2 ### for jwst ###

    ### create galfit feedme file ###
    if conf == True:
        cf = open(folder + f'{str(id)}_{str(filter)}_galfit.CONSTRAINTS', 'w+')
        cf.write('1      x    ' + str(-5) + '   ' + str(5) + ' \n') #pos x ±1 of input
        cf.write('1      y    ' + str(-5) + '   ' + str(5) + ' \n') #pos y ±1 of input
        cf.write('1      n    ' + str(0.05) + ' to  ' + str(10.0) + ' \n') #sersic index from 0.1 to 8.
        cf.write('1      mag    ' + str(-20.) + '   ' + str(20.) + ' \n') #±20 of input mag
        cf.write('1      re    ' + str(0.02) + ' to  ' + str(100) + ' \n') #half light radius
        cf.write('1      9    ' + str(0.02) + ' to  ' + str(1.0) + ' \n') # ellipticity
        cf.write('1      10    ' + str(-90.) + '   ' + str(90.) + ' \n') # position angle

        cf.close()
        constraints = f'{str(id)}_{str(filter)}_galfit.CONSTRAINTS'
    else:
        constraints = "none"

    gf = open(folder + f'{str(id)}_{str(filter)}_galfit.feedme', 'w+')
    ### these are the input filenames etc ####
    gf.write("# IMAGE and GALFIT CONTROL PARAMETERS\n")
    gf.write('A) '+ input_gfile +'          #input data image\n')
    gf.write('B) ' + output_gfile +'          #output image block\n')
    gf.write('C) ' + str(rms) + '          #sigma image\n')
    gf.write('D) '+str(psf_filename)+'          #Input PSF image (and diffusion kernel)\n')
    gf.write('E) 1        #PSF fine sampling factor\n')
    gf.write(f'F) {str(id)}_{str(filter)}_mask.fits           #mask image\n')
    gf.write(f'G) {str(constraints)}          #File with parameter constraints\n')
    gf.write('H) 1 101 1 101    	# Image region to fit (xmin xmax ymin ymax)\n') ### change to 1 60 1 60 for hst ###
    gf.write('I) 101 101          	# Size of the convolution box (x y)\n') ### change to 70, 70 for hst ###
    gf.write('J) ' + str(zp) + '          #Magnitude zero point\n')
    gf.write(f'K) {str(pixel_scale)}  {str(pixel_scale)}         	# Plate scale (dx dy)    [arcsec per pixel]\n') # 0.06  0.06 for hst
    gf.write('O) regular             	# Display type (regular, curses, both)\n')
    gf.write('P) 0                   	# Choose: 0=optimize, 1=model, 2=imgblock, 3=subcomps\n\n')

    gf.write('# Object number: 1 \n')
    gf.write('0) sersic          #object type\n')
    gf.write('1) ' + str(np.round(x,3)) + ' ' + str(np.round(y,3)) + ' 1' + ' 1          #position x,y\n')
    gf.write('3) ' + str(np.round(mag,3)) + ' 1          #Integrated magnitude\n')
    gf.write('4) ' + str(np.round(reff, 3)) + ' 1          # Half light radius in pixels\n')
    gf.write(f'5) {str(np.round(sersic,3))} 1          #Sersic index n\n')
    gf.write('6) 0 0\n')
    gf.write('7) 0 0\n')
    gf.write('8) 0 0\n')
    gf.write('9) ' + str(np.round(1-ratio,3)) + ' 1          #Axis ratio (b/a)\n') #change this back to 1 - ratio
    gf.write('10) ' + str(angle) + ' 1          #Position angle (PA) [deg: Up=0, Left=90]\n')
    gf.write('Z) 0          #output option\n\n')

    # make sky object in galfit file ###
    # Object number: 2
    gf.write('# Object number: 2 \n')
    gf.write('0) sky                    #  object type\n')
    gf.write('1) 0.0000      1          #  sky background at center of fitting region [ADUs]\n')
    gf.write('2) 0.0000      0          #  dsky/dx (sky gradient in x)\n')
    gf.write('3) 0.0000      0          #  dsky/dy (sky gradient in y)\n')
    gf.write('Z) 0                      #  output option (0 = resid., 1 = Dont subtract)\n\n')
    gf.close()




In [74]:


def perturb(value, scale=0.5, min_val=None, max_val=None):
    """Apply a small perturbation."""
    new_value = value * (1 + np.random.uniform(-scale, scale))
    if min_val is not None and max_val is not None:
        return np.clip(new_value, min_val, max_val)
    return new_value




In [70]:

def run_galfit(ids,path_to_stamp, filter, verb=True):
    ### first create all the stamps and mask files ###

    # stamp = batch_stamps(ids)
    stamp = path_to_stamp

    ### define paths to the folders/stamps and galfit executable ###
    

    galfit = "/Users/mhamadouche/Downloads/galfit-example/galfit "

    for j in ids: #tqdm(ids)
        # i = ids[0]
        path = f'/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/pysersic_files/{str(ids[0])}'
        # create_galfit_file(j, )
        galfit_file = f"{str(j)}_{str(filter)}_galfit.feedme"
        #for iteration in range(1):
        for i in range(3):
            filename = f"{path}/{str(j)}_{str(filter)}_galfit.{i:02d}"
            if os.path.exists(filename):
                galfit_file = filename
                counter = int(galfit_file.split(".")[-1])
                counter += 1

                modified_lines = []
                inside_object_1 = False  

                galfit_01 = f"{path}/{str(j)}_{str(filter)}_galfit.{i:02d}"  # Source file
                galfit_changed = f"{path}/{str(j)}_{str(filter)}_galfit.{counter:02d}"  # Target file

                with open(galfit_01, 'r') as gf:
                    #print("opened .01 file")
                    lines = gf.readlines()

                for line in lines:
                    if line.startswith('# Component number: 1'):
                        #print("Found Object 1")
                        inside_object_1 = True
                        modified_lines.append(line)
                        continue  # Skip to the next line to check inside object 1
                    # Modify lines within Object 1
                    elif line.startswith('# Component number: 2'):
                        break
                    if inside_object_1:
                        #print("Inside Object 1", inside_object_1)
                        if line.startswith(" 1) "):
                            #print(float(line.split()[1]), float(line.split()[2]))
                            x, y = float(line.split()[1]), float(line.split()[2])
                            modified_lines.append(f' 1) {np.round(x, 3)} {np.round(y, 3)} 1 1\n')
                        elif line.startswith(" 3) "):
                            mag = perturb(float(line.split()[1]), min_val=-20, max_val=20)
                            modified_lines.append(f' 3) {np.round(mag, 3)} 1\n')
                        elif line.startswith(" 4) "):
                            #print(float(line.split()[1]))
                            reff = perturb(float(line.split()[1]), min_val=0.1, max_val=100)
                            #print('new reff:', reff)
                            modified_lines.append(f' 4) {np.round(reff, 3)} 1\n')
                        elif line.startswith(" 5) "):
                            sersic = perturb(float(line.split()[1]), min_val=0.1, max_val=10)
                            modified_lines.append(f' 5) {np.round(sersic, 3)} 1\n')
                        elif line.startswith(" 9) "):
                            ratio = perturb(float(line.split()[1]), min_val=0.1, max_val=1.0)
                            modified_lines.append(f' 9) {np.round(1-ratio, 3)} 1\n')
                        elif line.startswith(" 10) "):
                            angle = perturb(float(line.split()[1]), min_val=-90, max_val=90)
                            modified_lines.append(f' 10) {np.round(angle, 3)} 1\n')
                        else:
                            modified_lines.append(line)

                    else:
                        modified_lines.append(line)

                # Write modified file as galfit.03
                with open(galfit_changed, 'w') as f:
                    f.writelines(modified_lines)

                galfit_file = galfit_changed

            else:
                counter = 0
                galfit_file = galfit_file

            os.chdir(f"{path}")

            sp = subprocess.run([galfit + galfit_file], shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,check=True)
            stdout = sp.stdout.decode('utf-8')

            # os.chdir(f"{path}/{str(j)}_galfit_files/")

        if verb:
            print(f"...... Galfitted object: {str(j)}_{str(filter)} ......")

        # os.chdir(f"/Users/mhamadou/Desktop/PhDStuff/Year4/")
        # logfile = f"{path}/{str(j)}_galfit_files/fit.log"
        # outcat(j, logfile, cols, cats, outfile = outfile_name)

    return 0



In [5]:
import astropy.io.fits as fits
import numpy as np
import os
wht = fits.open('/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/codes/test_/28812_f444_wht_cutout.fits')
folder = "/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/codes/test_/"
rms_map = 1/np.sqrt(wht[0].data)
id_ = '28812'
fits.writeto(os.path.join(folder, str(id_)+'_'+"rms.fits"), data = rms_map, overwrite = True)#*unit_to_uJy

In [7]:
mask_cutout = fits.open('/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/codes/test_/28812_f444_segmentation.fits')

ny,nx = 80,80
ic, jc = ny//2, nx//2
label = mask_cutout[0].data[ic, jc]

print(ic, jc)

mask_cutout = mask_cutout[0].data
mask = np.where(mask_cutout == int(label))
mask_cutout[mask] = 0

# fits.writeto(os.path.join(folder, str(id_)+'_'+"mask.fits"), data = mask_cutout, overwrite = True)

40 40


In [ ]:
id = 'rubies_uds_144195'
create_galfit_file(id, filter = 'F444W')


In [72]:

print("created stamps & galfit files: press enter to run galfit")
path_to_stamp = '/Users/mhamadouche/UMass/postdoc_work/LRDs_Project_2025:26/pysersic_files/rubies_uds_144195/rubies_uds_144195_F444W_stamp.fits'
ids = [id]
print("running galfit on sources")
run_galfit(ids, filter = 'F444W', path_to_stamp=path_to_stamp)


input("galfit run finished: press enter to extract best fits")

created stamps & galfit files: press enter to run galfit
running galfit on sources
...... Galfitted object: rubies_uds_144195_F444W ......


''